In [1]:
from RAG.RAG_query import *
import os

from RAG.RAG_query import get_topics_list
from config.loadConfig import load_project_config
from DB.qDrant import check_qdrant_status, QdrantConfig
from sentence_transformers import SentenceTransformer


# Connect to Qdrant
config = load_project_config(os.path.join(os.getcwd(), "config/config.yaml"))
qdrantConfig = QdrantConfig(
    host=config["db"]["qdrant"]["connection"]["host"],
    port=config["db"]["qdrant"]["connection"]["port"],
    timeout=config["db"]["qdrant"]["connection"]["timeout"]
)
status, client = check_qdrant_status(qdrantConfig)
embedding_model = SentenceTransformer(config['embedding_model'])
sparse_model = SparseTextEmbedding(model_name="Qdrant/bm25")
small_collection_name = config['db']['collections']['collection_small']['name']
large_collection_name = config['db']['collections']['collection_large']['name']

Connected. Collections: ['wiki_large_chunks', 'wiki_small_chunks']


In [21]:
import numpy as np

# Creating DB Of 10 points for each title
dense_vectors = []
labels = []
texts = []
topics_list = get_topics_list(client,1000)
np.random.shuffle(topics_list)
for i in topics_list:
    large_chunks = get_point_with_common_title(client, large_collection_name, i)
    small_chunks = []
    for j in large_chunks[0]:
        # print(j.id)
        # [print(i) for i in get_point_with_common_parent_id(client=client, collection_name=small_collection_name, parent_id=j.id)[0]]
        small_chunks.extend(get_point_with_common_parent_id(client, small_collection_name, j.id)[0])
    for k in range(0,10):
        labels.append(i)
        dense_vectors.append(small_chunks[k].vector["dense"])
        texts.append(small_chunks[k].payload["text"])
        print(small_chunks[k].payload["text"])

Bomber
Strategic bombing is done by heavy bombers primarily designed for long-range bombing missions against strategic targets such as supply bases, bridges, factories, shipyards, and cities themselves,  to diminish the enemy's ability to wage war by limiting access to resources through crippling infrastructure or reducing industrial output. Current examples include the strategic nuclear-armed bombers:
Tactical
During WWII with engine power as a major limitation, combined with the desire for accuracy and other operational factors, bomber designs tended to be tailored to specific roles. Early in the Cold War however, bombers were the only means of carrying nuclear weapons to enemy targets, and held the role of deterrence. With the advent of guided air-to-air missiles, bombers needed to avoid interception.
include the strategic nuclear-armed bombers: B-2 Spirit, B-52 Stratofortress, Tupolev Tu-95 'Bear', Tupolev Tu-22M 'Backfire' and Tupolev Tu-160 "Blackjack"; historically notable examp

In [25]:
print(labels[1])
print(texts[1])

Bomber
Tactical


In [16]:
import umap
import pandas as pd
import plotly.express as px

# Example metadata: replace with your own (e.g., labels, IDs, or other info)
# metadata = [f"Sample_{i}" for i in range(data.shape[0])]

# Run UMAP to reduce to 2D
umap_2d = umap.UMAP(n_components=2, random_state=42)
embedding_2d = umap_2d.fit_transform(dense_vectors)

# Create a DataFrame for Plotly
df = pd.DataFrame({
    'x': embedding_2d[:, 0],
    'y': embedding_2d[:, 1],
    'metadata': texts,
    'color': labels
})

# Create interactive scatter plot
fig = px.scatter(
    df,
    x='x',
    y='y',
    color='color',
    hover_data=['metadata'],  # Display metadata on hover
    title='Interactive UMAP 2D Projection'
)

# Optional: Add click interaction
fig.update_traces(
    marker=dict(size=8),
    selector=dict(mode='markers'),
    customdata=df[['metadata']].values  # Attach metadata for click events
)

# Add click event to display data
fig.update_layout(
    clickmode='event+select',
    showlegend=False
)

# Show the plot
fig.show()

# Optional: Add a callback for clicking (for Jupyter or Dash integration)
# This requires a Dash app or further customization for full click handling

/mnt/85e3eeea-3bda-4255-95a1-833fffd20e34/dev/RAG-Viz/.venv/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

